# Predicción de comportamientos con Word2vec

## Word2vec embeddings

### Extrayendo la secuencia de Comportamientos Colectivos de los Clientes

In [1]:
import pandas as pd
import gensim
from gensim import corpora, models, similarities

In [36]:
# data = pd.read_csv("../resultados/mobile_consume1_results.csv")
data = pd.read_csv("../../resultados/mobile_consume1_results.csv")
data.head()


,CO_ID,YEAR,WEEK,PROFILE_ID,SIZE,D0T0P0,D0T0P1,D0T0P2,D0T0P3,D0T0P4,...,D6T3P30,D6T3P31,D6T3P32,D6T3P33,D6T3P34,D6T3P35,D6T3P36,D6T3P37,INDIVIDUAL_CLUSTER,COLLECTIVE_CLUSTER
0,3132200,2018,18,0,2.5552,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,13
1,3132200,2018,19,1,3.3935,0.0057,0.0047,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,11
2,3132200,2018,20,2,4.0228,0.0005,0.0004,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,11
3,3132200,2018,21,3,5.8880,0.0839,0.0834,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,10
4,3132200,2018,22,4,3.6598,0.1549,0.1547,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,13


In [113]:
clientes = data.CO_ID.unique()

In [114]:
# CONSTRUYENDO VOCABULARIO COMPLETO DE CLIENTES

year_min = min(data.YEAR) 
year_max = max(data.YEAR) 
week_min = min(data.WEEK) 
week_max = max(data.WEEK) 
year_min = 2016

periodos = []

if year_min == year_max:
    year = year_min
    for week in range(week_min,week_max):
        print(year,week)
        
        weeks = ""
        if week<10:
            weeks = "0"+week
        periodos.append(str(year)+"-"+weeks)
        
else:
    for year in range(year_min,year_max):
        if year == year_min:
            for week in range(week_min,52):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        elif year == year_max-1:   
            for week in range(1,week_max):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        else:
            for week in range(1,52):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)


In [119]:
# Cargamos lista de palabras de los clientes 

data2 = pd.DataFrame(index = clientes, columns=periodos)
data2

,2016-18,2016-19,2016-20,2016-21,2016-22,2016-23,2016-24,2016-25,2016-26,2016-27,...,2017-22,2017-23,2017-24,2017-25,2017-26,2017-27,2017-28,2017-29,2017-30,2017-31
3132200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3273844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3768670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3877738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3960165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3973457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4000413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4004178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4010344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
tok_corp = []

for cli in clientes:
    data_i = data[(data.CO_ID == cli)]
    lista_i = list(data_i.COLLECTIVE_CLUSTER)

    for j in range(len(lista_i)):
        lista_i[j] = "cluster_"+str(lista_i[j])
    tok_corp.append(lista_i)
    

In [7]:
# tok_corp

### Modelo

In [8]:
model = gensim.models.Word2Vec(tok_corp, min_count = 1, size = 7)

In [31]:
# Diccionario

# sorted(model.wv.index2word)

In [11]:
model.most_similar("cluster_0")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_27', 0.8714534640312195),
 ('cluster_41', 0.8607761263847351),
 ('cluster_32', 0.8571873307228088),
 ('cluster_9', 0.8360229730606079),
 ('cluster_44', 0.8297167420387268),
 ('cluster_47', 0.8199203610420227),
 ('cluster_33', 0.8167862296104431),
 ('cluster_35', 0.790198802947998),
 ('cluster_59', 0.7651001811027527),
 ('cluster_19', 0.7608996629714966)]

In [12]:
import random

In [15]:
prueba = []

for i in range(3): 
    prueba.append("cluster_"+str(random.randint(0,69))+".0")
    
prueba = ['cluster_29',"cluster_0"]

In [16]:
model.predict_output_word(prueba, topn=5)

[('cluster_0', 0.12363096),
 ('cluster_41', 0.05700568),
 ('cluster_9', 0.04601787),
 ('cluster_27', 0.044365566),
 ('cluster_48', 0.04158786)]

In [17]:
model.predict_output_word(prueba, topn=5)

[('cluster_0', 0.12363096),
 ('cluster_41', 0.05700568),
 ('cluster_9', 0.04601787),
 ('cluster_27', 0.044365566),
 ('cluster_48', 0.04158786)]

In [24]:
# VOCABULARIO

print(sorted(list(model.wv.vocab.keys())))
print("Cantidad = "+str(len(model.wv.vocab)))

['cluster_0', 'cluster_1', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19', 'cluster_2', 'cluster_20', 'cluster_21', 'cluster_22', 'cluster_23', 'cluster_24', 'cluster_25', 'cluster_26', 'cluster_27', 'cluster_28', 'cluster_29', 'cluster_3', 'cluster_30', 'cluster_31', 'cluster_32', 'cluster_33', 'cluster_34', 'cluster_35', 'cluster_36', 'cluster_37', 'cluster_38', 'cluster_39', 'cluster_4', 'cluster_40', 'cluster_41', 'cluster_42', 'cluster_43', 'cluster_44', 'cluster_45', 'cluster_46', 'cluster_47', 'cluster_48', 'cluster_49', 'cluster_5', 'cluster_50', 'cluster_51', 'cluster_52', 'cluster_53', 'cluster_54', 'cluster_55', 'cluster_56', 'cluster_57', 'cluster_58', 'cluster_59', 'cluster_6', 'cluster_60', 'cluster_61', 'cluster_62', 'cluster_63', 'cluster_64', 'cluster_65', 'cluster_66', 'cluster_67', 'cluster_68', 'cluster_69', 'cluster_7', 'cluster_8', 'cluster_9']
Cantidad = 70


In [30]:
model["cluster_0.0"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-2.7467828 ,  1.202307  , -0.40277043, -2.4476228 , -2.8530068 ,
       -1.1204137 ,  0.9128713 ], dtype=float32)

In [25]:
# Referencias

# https://github.com/RaRe-Technologies/gensim/tree/develop/docs/notebooks
    